In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from scipy.spatial.distance import cdist
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import AffinityPropagation
from sklearn.cluster import Birch
from scipy.cluster.hierarchy import dendrogram , linkage

from sklearn.metrics import f1_score, confusion_matrix, accuracy_score

ModuleNotFoundError: No module named 'pandas'

In [ ]:
pip install 

In [ ]:
data = pd.read_csv('/Users/anastasiakucina/project/ML_course/data/archive/CC GENERAL.csv')
data.head()

## EASY

In [ ]:
# чтобы подобрать признаки для кластеризации, посмотрим на попарнуб корреляцию фичей 
# - чтобы лучше кластеризовать возьмем те, что меньше всего коррелируют 
intresting_columns= ['BALANCE', 'BALANCE_FREQUENCY', 'PURCHASES',
       'ONEOFF_PURCHASES', 'INSTALLMENTS_PURCHASES', 'CASH_ADVANCE',
       'PURCHASES_FREQUENCY', 'ONEOFF_PURCHASES_FREQUENCY',
       'PURCHASES_INSTALLMENTS_FREQUENCY', 'CASH_ADVANCE_FREQUENCY',
       'CASH_ADVANCE_TRX', 'PURCHASES_TRX', 'CREDIT_LIMIT', 'PAYMENTS',
       'MINIMUM_PAYMENTS', 'PRC_FULL_PAYMENT', 'TENURE']

In [ ]:
data[['BALANCE', 'BALANCE_FREQUENCY', 'PURCHASES',
       'ONEOFF_PURCHASES', 'INSTALLMENTS_PURCHASES', 'CASH_ADVANCE',
       'PURCHASES_FREQUENCY', 'ONEOFF_PURCHASES_FREQUENCY',
       'PURCHASES_INSTALLMENTS_FREQUENCY', 'CASH_ADVANCE_FREQUENCY',
       'CASH_ADVANCE_TRX', 'PURCHASES_TRX', 'CREDIT_LIMIT', 'PAYMENTS',
       'MINIMUM_PAYMENTS', 'PRC_FULL_PAYMENT', 'TENURE']].corr(method='kendall')

In [ ]:
intresting_columns= ['BALANCE', 'PURCHASES',
       'INSTALLMENTS_PURCHASES',
       'ONEOFF_PURCHASES_FREQUENCY',
       'PURCHASES_INSTALLMENTS_FREQUENCY', 'PURCHASES_TRX', 'CREDIT_LIMIT', 'PAYMENTS',
       'TENURE']

In [ ]:
for i in range(len(intresting_columns)):
    for j in range(i+1, len(intresting_columns)):
        sns.lineplot(data=data, x=intresting_columns[i], y=intresting_columns[j])
        sns.scatterplot(data=data, x=intresting_columns[i], y=intresting_columns[j])
        plt.show()

##### Микро-вывод

- Среди оставшихся фичей нет сильной корреляции, поэтому имеет смысл кластеризовать по ним -- но для лучшей кластеризации нужно будет попробовать разные методы, сделаем это в следующем задании :)

In [ ]:
# first try of claterization
def distance_by_n(x, n):
    model = KMeans(n_clusters=n)
    model.fit(x)
    return model.inertia_

In [ ]:
data.info()

In [ ]:
data.dropna(inplace = True)

In [ ]:
distances = [distance_by_n(data[intresting_columns], n) for n in range(1, 10)]
plt.plot(range(1, 10), distances)
plt.title('Elbow method')

In [ ]:
kmeans = KMeans(n_clusters=4, random_state=0)
clusters = kmeans.fit_predict(data[intresting_columns])

# Визуализируем результаты кластеризации по двум признакам
for i in range(len(intresting_columns)):
    for j in range(i+1, len(intresting_columns)):
        plt.scatter(data[intresting_columns[i]], data[intresting_columns[j]], c=clusters, cmap='viridis')
        plt.xlabel('Amount of %s' % intresting_columns[i])
        plt.ylabel('Amount of %s' % intresting_columns[j])
        plt.show()

#### Вывод на глаз:

- по большинству парных графиков видно, что разделение на кластеры неплохое - если смотреть по диагоняли снизу слева -> наверх вправо 

## MEDIUM

In [ ]:
methods = {
    'DBSCAN' : DBSCAN(eps=5, min_samples=100),
    'AffinityPropagation' : AffinityPropagation(random_state=42),
    'Birch' : Birch(n_clusters=None)
}

In [ ]:
best_model = None
worst_model = None
best_accuracy = 0
worst_accuracy = 1

for name, model in methods.items():
    model.fit_predict(data[intresting_columns])
    # Визуализируем результаты кластеризации по двум признакам
    for i in range(len(intresting_columns)):
        for j in range(i+1, len(intresting_columns)):
            plt.scatter(data[intresting_columns[i]], data[intresting_columns[j]], c=clusters, cmap='viridis')
            plt.xlabel('Amount of %s' % intresting_columns[i])
            plt.ylabel('Amount of %s' % intresting_columns[j])
            plt.show()
    